In [88]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets as ds
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [89]:
# 0) prepare data
bc = ds.load_breast_cancer()
X,Y = bc.data, bc.target

n_samples, n_features = X.shape
print(n_samples, n_features)

569 30


In [90]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=1234)

In [91]:
# scale, make feature has 0 means
sc = StandardScaler()
print(X_train[0])
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
print('after fit')
print(X_train[0])

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
Y_train = torch.from_numpy(Y_train.astype(np.float32))
Y_test = torch.from_numpy(Y_test.astype(np.float32))

#shape y from horiz to vertical
Y_train = Y_train.view(Y_train.shape[0],1)
Y_test = Y_test.view(Y_test.shape[0],1)

[1.288e+01 1.822e+01 8.445e+01 4.931e+02 1.218e-01 1.661e-01 4.825e-02
 5.303e-02 1.709e-01 7.253e-02 4.426e-01 1.169e+00 3.176e+00 3.437e+01
 5.273e-03 2.329e-02 1.405e-02 1.244e-02 1.816e-02 3.299e-03 1.505e+01
 2.437e+01 9.931e+01 6.747e+02 1.456e-01 2.961e-01 1.246e-01 1.096e-01
 2.582e-01 8.893e-02]
after fit
[-0.36180827 -0.26521011 -0.31715702 -0.46713841  1.80382609  1.18174184
 -0.51689239  0.10653677 -0.39005152  1.39140136  0.14370971 -0.12075458
  0.16013008 -0.13255126 -0.58631633 -0.12476339 -0.57865272  0.10906824
 -0.28187354 -0.1889237  -0.25710149 -0.24033176 -0.2441722  -0.36688232
  0.5448709   0.24812607 -0.71088618 -0.07967528 -0.52798733  0.2506337 ]


In [65]:
# 1) model
# f = wx+b, sigmoid at the end

class LogisticRegression(nn.Module):    
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
        
    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted
        
model = LogisticRegression(n_features)


In [66]:
# 2) loss and optimizer
#Binary Cross Entropy loss ln = -wn[yn * log(xn) + (1-yn)*log(1-xn)]
learning_rate = 0.02
criterion = nn.BCELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [69]:
# 3) training
num_epochs = 100
for epoch in range(num_epochs):
    # forward pass and loss
    y_predicted = model(X_train)
    loss = criterion(y_predicted, Y_train)
    
    # backward pass
    loss.backward()
    
    #updates
    optimizer.step()
    
    #zero gradients
    optimizer.zero_grad
    
    if(epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')
    

epoch: 10, loss = 0.2442
epoch: 20, loss = 0.2441
epoch: 30, loss = 0.2438
epoch: 40, loss = 0.2433
epoch: 50, loss = 0.2426
epoch: 60, loss = 0.2420
epoch: 70, loss = 0.2416
epoch: 80, loss = 0.2412
epoch: 90, loss = 0.2410
epoch: 100, loss = 0.2409


In [72]:
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(Y_test).sum() / float(Y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

accuracy = 0.9474
